In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Install dependencies

In [ ]:
!pip install -q transformers

# 2. Run sentiment analysis predictions by using Pipeline

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

# tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
print(csvreader)
csv_reader = csvreader
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [ ]:
import pandas as pd

In [ ]:
def process_the_script(important_chars,txt_path):
    df = pd.read_csv(txt_path)
    char_Wise_Dialogue = {}
    for char in important_chars:
        char_Wise_Dialogue[char] = []
    for i in range(len(df)):
        for char in important_chars:
            if char in df['character'][i]:
                char_Wise_Dialogue[char].append(df['line'][i])
    return char_Wise_Dialogue


In [ ]:
def sentiment_cardiff(text):
  c = csvreader
  # mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
  # with urllib.request.urlopen(mapping_link) as f:
  #     html = f.read().decode('utf-8').split("\n")
  #     csvreader = csv.reader(html, delimiter='\t')
  # labels = [row[1] for row in csvreader if len(row) > 1]
  labels = ['negative', 'neutral', 'positive']
  # print(labels)
  # PT
  model = AutoModelForSequenceClassification.from_pretrained(MODEL)
  model.save_pretrained(MODEL)
  text =  text
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  # print(ranking)
  for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
      return l
      # print(f"{i+1}) {l} {np.round(float(s), 4)}")
sentiment_cardiff('Balls in my face')


'negative'

In [ ]:
char_movie_wise_dialogues = {}
def compute_sentiment_labels(dialogues):
    sid = sentiment_cardiff(dialogues)
    sentiment_labels=[]
    for i in sid:
      if i['label'] == 'positive':
        sentiment_labels.append(0.09)
      elif i['label'] == 'negative':
        sentiment_labels.append(-0.09)
      else:
        sentiment_labels.append(0.01)


def plot_sentiment_distribution(sent_dicts):
    pos_counts = {}
    neg_counts = {}
    neu_counts = {}

    for char in sent_dicts:
        pos_counts[char] = 0
        neg_counts[char] = 0
        neu_counts[char] = 0
        for sent in sent_dicts[char]:
            if sent >= 0.05:
                pos_counts[char] += 1
            elif sent <= -0.05:
                neg_counts[char] += 1
            else:
                neu_counts[char] += 1
    # plot graph for sentiment analysis
    import matplotlib.pyplot as plt
    import numpy as np

    chars = list(pos_counts.keys())
    pos = list(pos_counts.values())
    neg = list(neg_counts.values())
    neu = list(neu_counts.values())

    x = np.arange(len(chars))
    fig, ax = plt.subplots()
    bar_width = 0.2
    b1 = ax.bar(x, pos, width=bar_width, label='Number of Positive Dialogues')
    b2 = ax.bar(x + bar_width, neg, width=bar_width, label='Number of Negative Dialogues')
    b3 = ax.bar(x + 2*bar_width, neu, width=bar_width, label='Number of Neutral Dialogues')

    ax.set_xticks(x + bar_width)
    ax.set_xticklabels(chars, rotation=90)
    ax.legend()
    plt.show()
    total = {}
    for char in sent_dicts:
        total[char] = pos_counts[char] + neg_counts[char] + neu_counts[char]
    pos_perc = {}
    neg_perc = {}
    neu_perc = {}
    for char in sent_dicts:
        pos_perc[char] = pos_counts[char] / total[char]
        neg_perc[char] = neg_counts[char] / total[char]
        neu_perc[char] = neu_counts[char] / total[char]

    # plot graph for sentiment analysis
    import matplotlib.pyplot as plt
    import numpy as np

    chars = list(pos_perc.keys())
    pos = list(pos_perc.values())
    neg = list(neg_perc.values())
    neu = list(neu_perc.values())

    x = np.arange(len(chars))
    fig, ax = plt.subplots()
    bar_width = 0.2
    b1 = ax.bar(x, pos, width=bar_width, label='Percent of Positive Dialogues')
    b2 = ax.bar(x + bar_width, neg, width=bar_width, label='Percent of Negative Dialogues')
    b3 = ax.bar(x + 2*bar_width, neu, width=bar_width, label='Percent of Neutral Dialogues')
    ax.set_xticks(x + bar_width)
    ax.set_xticklabels(chars, rotation=90)
    ax.legend()
    plt.show()
    del sent_dicts

def plot_movie_sentiments(movie_path,important_chars):
    dialogues = process_the_script(important_chars,movie_path)
    sent_dicts = {}
    for char in dialogues:
        sent_dicts[char] = compute_sentiment_labels(dialogues[char])

    char_movie_wise_dialogues[movie_path.split("/")[1]] = sent_dicts
    plot_sentiment_distribution(sent_dicts)
    del important_chars,sent_dicts,dialogues



### Iron man 1 (2008)

In [ ]:
read_data = '/content/drive/MyDrive/NLP PROJECT/script_csv/iron_man.csv'
important_chars = ['PEPPER POTTS','PHIL COULSON','TONY STARK','OBADIAH','CHRISTINE EVERHART','JARVIS','JAMES RHODES','HAPPY HOGAN','HO YINSEN']
plot_movie_sentiments(read_data,important_chars)

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
%pip install textblob
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer,PatternAnalyzer

1) negative 0.4995
2) neutral 0.4575
3) positive 0.043


## MODEL FOR TASK 4

In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.1 MB/s eta 0:00:00


In [ ]:
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
csv_iron_man = '/content/drive/MyDrive/NLP PROJECT/script_csv/avengers_endgame.csv'
male_iron = []
female_iron = []
df = pd.read_csv(csv_iron_man)
for index, row in df.iterrows():
    if row['gender'] == 'MALE':
        male_iron.append(row['line'])
    elif row['gender'] == 'FEMALE':
        female_iron.append(row['line'])
print(len(male_iron),len(female_iron))

840 187


In [ ]:
pipe = pipeline('text-classification', model='padmajabfrl/Gender-Classification')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm
predictions =[]
for i in tqdm(male_iron+female_iron):
  predictions.append(pipe([i]))


100%|██████████| 1027/1027 [01:16<00:00, 13.38it/s]


In [ ]:
print(predictions)

[[{'label': 'Male', 'score': 0.5448513627052307}], [{'label': 'Female', 'score': 0.7609208226203918}], [{'label': 'Female', 'score': 0.8630147576332092}], [{'label': 'Female', 'score': 0.602654755115509}], [{'label': 'Female', 'score': 0.7001383900642395}], [{'label': 'Female', 'score': 0.5840664505958557}], [{'label': 'Male', 'score': 0.7777964472770691}], [{'label': 'Female', 'score': 0.942547619342804}], [{'label': 'Male', 'score': 0.7255149483680725}], [{'label': 'Female', 'score': 0.6033810973167419}], [{'label': 'Female', 'score': 0.8552485704421997}], [{'label': 'Male', 'score': 0.917543351650238}], [{'label': 'Female', 'score': 0.76805180311203}], [{'label': 'Female', 'score': 0.6957595348358154}], [{'label': 'Female', 'score': 0.9000706672668457}], [{'label': 'Male', 'score': 0.7217594385147095}], [{'label': 'Male', 'score': 0.982718825340271}], [{'label': 'Female', 'score': 0.7528310418128967}], [{'label': 'Male', 'score': 0.9999997615814209}], [{'label': 'Male', 'score': 0.9

In [ ]:
pred_labels = [i[0]['label'] for i in predictions]
true_labels = ['Male' for i in range(840)]
for i in range(187):
  true_labels.append('Female')

# import classification report
from sklearn.metrics import classification_report

creport = classification_report(pred_labels,true_labels)
print(creport)

              precision    recall  f1-score   support

      Female       0.50      0.17      0.25       568
        Male       0.44      0.80      0.56       459

    accuracy                           0.45      1027
   macro avg       0.47      0.48      0.41      1027
weighted avg       0.47      0.45      0.39      1027



In [ ]:
pipe2 = pipeline('text-classification', model='Saulr/distilbert-base-uncased-finetuned-gender-classification')


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm
predictions =[]
for i in tqdm(male_iron+female_iron):
  predictions.append(pipe2([i]))
pred_labels = [i[0]['label'] for i in predictions]


100%|██████████| 834/834 [00:56<00:00, 14.69it/s]


In [ ]:
print(predictions)
pred_labels = [i[0]['label'] for i in predictions]
true_labels = ['LABEL_1' for i in range(684)]
for i in range(150):
  true_labels.append('LABEL_0')

# import classification report
from sklearn.metrics import classification_report

creport = classification_report(pred_labels,true_labels)
print(creport)

[[{'label': 'LABEL_1', 'score': 0.8333889245986938}], [{'label': 'LABEL_1', 'score': 0.7337230443954468}], [{'label': 'LABEL_1', 'score': 0.8985379338264465}], [{'label': 'LABEL_1', 'score': 0.702217161655426}], [{'label': 'LABEL_1', 'score': 0.954342246055603}], [{'label': 'LABEL_0', 'score': 0.7117922306060791}], [{'label': 'LABEL_0', 'score': 0.9696023464202881}], [{'label': 'LABEL_1', 'score': 0.751087486743927}], [{'label': 'LABEL_1', 'score': 0.982453465461731}], [{'label': 'LABEL_1', 'score': 0.9662669897079468}], [{'label': 'LABEL_1', 'score': 0.8332409262657166}], [{'label': 'LABEL_0', 'score': 0.9503487944602966}], [{'label': 'LABEL_1', 'score': 0.8941092491149902}], [{'label': 'LABEL_1', 'score': 0.6062026619911194}], [{'label': 'LABEL_1', 'score': 0.8572000861167908}], [{'label': 'LABEL_0', 'score': 0.8136311769485474}], [{'label': 'LABEL_0', 'score': 0.934401273727417}], [{'label': 'LABEL_0', 'score': 0.7340652346611023}], [{'label': 'LABEL_0', 'score': 0.994682252407074}]

In [ ]:
pipe3 = pipeline('text-classification', model='priyabrat/UpTag_GenderClassification')


config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/268M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at priyabrat/UpTag_GenderClassification.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm
predictions =[]
for i in tqdm(male_iron+female_iron):
  predictions.append(pipe2([i]))
pred_labels = [i[0]['label'] for i in predictions]

100%|██████████| 834/834 [01:02<00:00, 13.34it/s]


In [ ]:
print(predictions)
pred_labels = [i[0]['label'] for i in predictions]
true_labels = ['LABEL_1' for i in range(684)]
for i in range(150):
  true_labels.append('LABEL_0')

# import classification report
from sklearn.metrics import classification_report

creport = classification_report(pred_labels,true_labels)
print(creport)

[[{'label': 'LABEL_1', 'score': 0.8333889245986938}], [{'label': 'LABEL_1', 'score': 0.7337230443954468}], [{'label': 'LABEL_1', 'score': 0.8985379338264465}], [{'label': 'LABEL_1', 'score': 0.702217161655426}], [{'label': 'LABEL_1', 'score': 0.954342246055603}], [{'label': 'LABEL_0', 'score': 0.7117922306060791}], [{'label': 'LABEL_0', 'score': 0.9696023464202881}], [{'label': 'LABEL_1', 'score': 0.751087486743927}], [{'label': 'LABEL_1', 'score': 0.982453465461731}], [{'label': 'LABEL_1', 'score': 0.9662669897079468}], [{'label': 'LABEL_1', 'score': 0.8332409262657166}], [{'label': 'LABEL_0', 'score': 0.9503487944602966}], [{'label': 'LABEL_1', 'score': 0.8941092491149902}], [{'label': 'LABEL_1', 'score': 0.6062026619911194}], [{'label': 'LABEL_1', 'score': 0.8572000861167908}], [{'label': 'LABEL_0', 'score': 0.8136311769485474}], [{'label': 'LABEL_0', 'score': 0.934401273727417}], [{'label': 'LABEL_0', 'score': 0.7340652346611023}], [{'label': 'LABEL_0', 'score': 0.994682252407074}]